# Top Performers Analysis

This notebook looks at the data for all funds, trying to find the current top funds by ranking the funds by using a custom ranking algorithm.

Note: This notebook assumes that you have loaded data within the `data/portfolios.json` and `data/pricing` by running the `init.py`.


In [53]:
# Lib imports.
import pandas as pd
import numpy as np
import json
import os
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.expected_returns import mean_historical_return

# Local imports.
from strategies.augmentation import augment_df
from strategies.ranking import rank_funds

In [54]:
# Constants
DATA_PATH = "data/pricing"
FUNDS_PATH = "data/portfolios.json"

# Columns.
COLUMN_DATE = "asOfDate"

## Initial Load

Loads all the data necessary to access funds.


In [55]:
# Get all the funds from the file.
with open(FUNDS_PATH, "r") as f:
    funds = json.load(f)

# Convert to ID to fund.
funds_map = {f["portId"]: f for f in funds}

print(funds_map)

{'8617': {'allocation': 'Bonds 0%,Cash 0%,Shares 100%', 'annualReturns': [{'value': '+18.95%', 'label': '01 Oct 2023 - 30 Sep 2024'}, {'value': '+9.96%', 'label': '01 Oct 2022 - 30 Sep 2023'}, {'value': '-4.65%', 'label': '01 Oct 2021 - 30 Sep 2022'}, {'value': '+23.09%', 'label': '01 Oct 2020 - 30 Sep 2021'}, {'value': '+4.38%', 'label': '01 Oct 2019 - 30 Sep 2020'}], 'annualReturnData': {}, 'assetClass': 'equity', 'domicileType': 'gb-domicile', 'esgType': '', 'fundCategory': 'buildMyOwn', 'id': 'vanguard-ftse-global-all-cap-index-fund-gbp-acc', 'ids': {}, 'inceptionDate': '2016-11-08', 'inceptionDates': {}, 'managementType': 'Index', 'name': 'FTSE Global All Cap Index Fund', 'noaValue': '7516', 'ocfValue': 0.23, 'ocfValues': {}, 'portId': '8617', 'parentPortId': '8617', 'productType': 'mf', 'readyMadeType': '', 'regionType': 'Global', 'riskLevel': '5', 'shareClass': 'Accumulation', 'sedol': 'BD3RZ58', 'fundGroupHedgedFunds': False, 'closeIndicator': False, 'upcomingChanges': False}, 

## Augmentation

Since the data is not complete and is missing some indexes, we need to make sure to add that data manually.

We are adding the following metrics:

- **1-Year and 5-Year Return.** The 1-year return and 5-year return measure the percentage change in the price over the past 1 year and 5 years, respectively.
  ![Formula for 1-Year/5-Year returns.](./docs/augmentation-1.png)

- **Expene Ratio.** The expense ratio is typically provided by the fund issuer and is not directly calculable from price data. It represents the annual management fees charged by the fund, expressed as a percentage of total assets. In this case we are taking this from the data in Vanguard's API.

- **Volatility (Standard Deviation of Returns).** Volatility is commonly measured by the standard deviation of daily returns. You can calculate the daily returns and then compute their standard deviation over a given period. Multiply by SQRT(252) to annualize the volatility (since there are ~252 trading days in a year).
  ![Formula for volatility.](./docs/augmentation-2.png)

- **Sharpe Ratio.** The Sharpe ratio measures risk-adjusted returns. It compares the fund’s excess return (return above the risk-free rate) to its volatility. The risk-free rate is the return of a virtually risk-free investment, like government bonds (you can assume around 0.5% to 2% depending on the market environment).
  ![Formula for sharpe ratio.](./docs/augmentation-3.png)


In [56]:
# Building out all the data into a map where {FUND_ID: {pricing: prices, data: JSON_DATA}}
funds_performance = {
    fid: {
        "pricing": pd.read_csv(os.path.join(DATA_PATH, f"{fid}.csv")).sort_values(
            by="asOfDate", ascending=True
        ),
        "data": funds_map[fid],
    }
    for fid in funds_map
}

In [57]:
# Computes a single DF with the fund statistics, with the fund name
fund_statistics = pd.DataFrame(
    [
        {
            "Name": funds_performance[fid]["data"]["name"],
            "Type": funds_performance[fid]["data"]["shareClass"],
            "id": fid,
            "Expense Ratio": funds_performance[fid]["data"]["ocfValue"],
            **augment_df(funds_performance[fid]["pricing"], "price").iloc[-1].to_dict(),
        }
        for fid in funds_performance
    ]
)

## Computing Performance

This section computes all the performance on a daily basis, for all the funds.


In [58]:
# Filter for columns.
df_filter = ["Name", "Type", "Total Score"]
LIMIT = 15

# Do the ranking computation, dedupe by fund type using the name.
ranked_funds_complete = rank_funds(fund_statistics)
idx = ranked_funds_complete.groupby("Name")["Total Score"].idxmax()
ranked_funds = ranked_funds_complete.loc[idx].sort_values(
    by="Total Score", ascending=False
)

top_ranking_funds = ranked_funds.head(LIMIT)
print("Top Ranked\n", top_ranking_funds[df_filter], "\n\n")

Top Ranked
                                                Name          Type  Total Score
127                               S&P 500 UCITS ETF  Accumulation     0.813335
121                    FTSE North America UCITS ETF  Accumulation     0.809782
148             ESG North America All Cap UCITS ETF  Accumulation     0.792753
116                  FTSE Developed World UCITS ETF  Accumulation     0.788136
120                        FTSE All-World UCITS ETF  Accumulation     0.775724
67                     ESG Global All Cap UCITS ETF  Accumulation     0.765774
95   USD Emerging Markets Government Bond UCITS ETF  Accumulation     0.703546
118    FTSE All-World High Dividend Yield UCITS ETF  Accumulation     0.689242
93            USD Corporate 1-3 Year Bond UCITS ETF  Accumulation     0.673568
142          ESG Developed Europe All Cap UCITS ETF  Accumulation     0.664463
119                 FTSE Emerging Markets UCITS ETF  Accumulation     0.654474
146          ESG Emerging Markets All Ca

In [59]:
print("Bottom Rank\n", ranked_funds.tail(LIMIT)[df_filter], "\n\n")

Bottom Rank
                                                  Name                 Type  \
53   U.K. Short-Term Investment Grade Bond Index Fund         Accumulation   
144      ESG Developed Asia Pacific All Cap UCITS ETF         Accumulation   
90                                 FTSE 100 UCITS ETF         Accumulation   
2                           FTSE 100 Index Unit Trust         Accumulation   
140                SustainableLife 80-90% Equity Fund         Accumulation   
115                              FTSE Japan UCITS ETF         Accumulation   
17                    U.K. Government Bond Index Fund         Accumulation   
7                    Japan Government Bond Index Fund  Hedged Accumulation   
89                            Active U.K. Equity Fund         Accumulation   
97                                U.K. Gilt UCITS ETF         Accumulation   
41              U.K. Inflation-Linked Gilt Index Fund         Accumulation   
13                             Japan Stock Index Fu

In [60]:
print(
    "Unranked\n",
    ranked_funds[ranked_funds["Total Score"].isnull()][df_filter],
    "\n\n",
)

Unranked
 Empty DataFrame
Columns: [Name, Type, Total Score]
Index: [] 




## Portfolio Optimisation

This section uses a bunch of libraries to optimise the portfolio.

In [61]:
# Get the minimum date across all of them.
min_date = max([min(funds_performance[id]["pricing"]["asOfDate"]) for id in top_ranking_funds["id"]])

# Concat all of them.
complete_top_ranking = None

for id in top_ranking_funds["id"]:
    __df : pd.DataFrame = funds_performance[id]["pricing"]
    __df = __df[__df["asOfDate"] >= min_date][["asOfDate", "price"]]
    __df = __df.rename(
        columns={
            "price": f"{funds_performance[id]['data']['name']} ({funds_performance[id]['data']['shareClass']})"
        }
    ).set_index("asOfDate")
    complete_top_ranking = (
        __df if complete_top_ranking is None else complete_top_ranking
    ).combine_first(__df)

complete_top_ranking

,ESG Developed Europe All Cap UCITS ETF (Accumulation),ESG Developed World All Cap Equity Index Fund (Accumulation),ESG Emerging Markets All Cap UCITS ETF (Accumulation),ESG Global All Cap UCITS ETF (Accumulation),ESG North America All Cap UCITS ETF (Accumulation),FTSE All-World High Dividend Yield UCITS ETF (Accumulation),FTSE All-World UCITS ETF (Accumulation),FTSE Developed Europe UCITS ETF (Accumulation),FTSE Developed Europe ex UK UCITS ETF (Accumulation),FTSE Developed World UCITS ETF (Accumulation),FTSE Emerging Markets UCITS ETF (Accumulation),FTSE North America UCITS ETF (Accumulation),S&P 500 UCITS ETF (Accumulation),USD Corporate 1-3 Year Bond UCITS ETF (Accumulation),USD Emerging Markets Government Bond UCITS ETF (Accumulation)
asOfDate,,,,,,,,,,,,,,,
2022-10-11,4.3218,325.4315,5.0000,4.0528,4.1002,52.2242,87.9132,32.0627,32.2579,68.2050,47.2434,86.3847,66.3690,51.1376,44.9782
2022-10-12,4.2938,324.9418,4.9952,4.0426,4.0925,52.0380,87.6365,31.8977,32.1351,67.9721,47.2147,86.1520,66.1521,51.1320,44.9499
2022-10-13,4.3205,325.5625,4.9399,4.0979,4.1877,52.8611,88.9284,32.1565,32.2980,69.1826,46.7027,88.2916,67.8773,50.9773,44.6576
2022-10-14,4.3536,320.4910,4.9917,4.0508,4.0874,52.7253,87.9461,32.3330,32.5459,68.2698,47.0853,86.2053,66.2734,50.9325,44.5757
2022-10-17,4.4353,323.1689,5.0092,4.1406,4.2093,53.4876,89.7385,32.9242,33.1384,69.7959,47.2581,88.5553,68.0280,50.9852,44.6411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-07,6.1738,438.9863,7.1164,6.3917,6.6722,76.4049,138.7154,45.6269,46.4221,108.9388,67.7675,140.1250,108.0480,57.3464,56.5702
2024-10-08,6.1584,441.1903,6.9785,6.4128,6.7462,75.7086,138.8254,45.3912,46.2743,109.3451,66.1853,141.3828,109.0952,57.3808,56.5678
2024-10-09,6.2013,444.6239,6.9199,6.4433,6.7964,75.8724,139.4170,45.6958,46.5760,109.9770,65.5739,142.4004,109.8736,57.3472,56.5300


In [65]:
# Calculate expected returns and covariance matrix
mu = mean_historical_return(complete_top_ranking)
cov_matrix = CovarianceShrinkage(complete_top_ranking).ledoit_wolf()

# Optimize for maximum Sharpe ratio
ef = EfficientFrontier(mu, cov_matrix)
weights = ef.max_sharpe()

# Clean weights and display portfolio allocation
cleaned_weights = ef.clean_weights()
ef.portfolio_performance(verbose=True)

# Display recommended allocation
for item, val in cleaned_weights.items():
    if val > 0:
        print(item, "-->", val)

Expected annual return: 10.7%
Annual volatility: 3.5%
Sharpe Ratio: 2.46
FTSE Developed Europe UCITS ETF (Accumulation) --> 0.06306
FTSE Emerging Markets UCITS ETF (Accumulation) --> 0.05364
S&P 500 UCITS ETF (Accumulation) --> 0.11815
USD Corporate 1-3 Year Bond UCITS ETF (Accumulation) --> 0.63912
USD Emerging Markets Government Bond UCITS ETF (Accumulation) --> 0.12604
